In [1]:
from keras import layers, models
from keras import initializers , preprocessing, utils
import numpy as np
import re
import math
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# Alphabet
# "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
letters = "abcdefghijklmnopqrstuvwxyz"
numbers = "0123456789"
other_char = ",;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}" # original: -,;.!?:’’’/\|_@#$%ˆ&* ̃‘+-=<>()[]{}, changed: -,;.!?:’/\|_@#$%ˆ&* ̃‘+-=<>()[]{}
new_line = "\n"

alphabet = letters + numbers + other_char + new_line
print(alphabet)
print(len(alphabet))

abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'"/\|_@#$%^&*~`+-=<>()[]{}

69


In [3]:
# type of text
# __label__2 This is my first PDA/Organizer: I purchased this about 4 months ago and it really is easy to use especially if you are familiar with Microsoft Word and Excel. I use it to copy files from my PC that I want to have handy. The only thing I wish it had is a backlight. But that hasn't come into play too often. I usually have to charge it every 2 or 3 days. I recommend this product if you are not worried about having a lot accessories to go with it, because they are not that many available.
# __label__1 Piece of Crap: I have ordered thousands of items in my lifetime, and bar none, this is the biggest piece of crap I have ever received. It is supposed to come completely put together, but when it arrived, there were no less than 10 little parts that had come off the screen assembly and one screen that was loose and all of its parts dangling. The first thing that happenned is that I cut myself on one of the screens because the protective side had come off. I was bleeding for quite some time. The second thing I noticed was the extremely poor quality of the material and assembly. I am very handy by nature, but even I had to give up trying to put this piece of junk back together again. Plus it was not worth it, even if I had put it together again, all I would have then had was a piece of crap fireplace screen.
# data from  https://www.kaggle.com/bittlingmayer/amazonreviews

dir_name = './data/'
test_file_name = 'test.ft.txt'
train_file_name = 'train.ft.txt'


def getListOfLabelsAndLinesFromFile(file_name): # label 1 = 0, label 2 = 1
    
    labels = []
    lines = []
    total_labels = []
    count = 0
    file = open(file_name, 'r')
    for line in file:
        match = re.match('(__label__([0-9])) (.*)', line)
        if match:
            label = int(match.group(2)) - 1 # to make the label start in 0 (ZERO)
            text = match.group(3)
            if not label in total_labels: total_labels.append(label)
            labels.append(label)
            lines.append(text)
        else:
            print('---- ERROR ----')
        count += 1
    file.close()
    return { 'labels': labels, 'lines': lines, 'total_labels': total_labels, 'total_lines': count }

train_data_info = getListOfLabelsAndLinesFromFile(dir_name + train_file_name)
test_data_info = getListOfLabelsAndLinesFromFile(dir_name + test_file_name)

print(test_data_info['total_lines'])
print(train_data_info['total_lines'])
# ratio 9 

400000
3600000


In [4]:
# Constants
# Embedding
max_len_text = 1014
max_features_for_embedding = len(alphabet)
output_from_embedding = 128

# Large Model
initializer_large = { 'mean': 0.0, 'stddev': 0.02}
activation_conv = 'relu'
conv_output_dim = 256
pool_size = 3
kernel_size_7 = 7 # first_WITH_maxpool
kernel_size_3 = 3 # intermediate_NO_maxpool

# Small model
initializer_small = { 'mean': 0.0, 'stddev': 0.05}
activation_dens = 'relu'
dropout_rate = 0.5

In [5]:
# Create Model
# https://medium.com/@romannempyre/sentiment-analysis-using-1d-convolutional-neural-networks-part-1-f8b6316489a2
# https://github.com/chaitjo/character-level-cnn
# https://missinglink.ai/guides/deep-learning-frameworks/keras-conv1d-working-1d-convolutional-neural-networks-keras/
# https://blog.goodaudience.com/introduction-to-1d-convolutional-neural-networks-in-keras-for-time-sequences-3a7ff801a2cf?gi=5c4a324fc922
# https://medium.com/@bramblexu/character-level-cnn-with-keras-50391c3adf33

model = models.Sequential() 

#model.add(layers.Input(shape=(max_len_text,),  dtype='int32'))

# AS the one hot approach, leave us with a very sparse and high dimensional matrix, we apply embedding
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs
# Embedding ( 
#    max_features/input_dim = we got just 70 different types of inputs / Size of the vocabulary, 
#    output_dim = learn N dimensional embeddings for each of the input_dim words/characters,
#    input_length needed to flatten = cut the words/characters of each element to that lenght so the max
#                                     qty of elements in each phrase will be that one
# )    
model.add(layers.Embedding(max_features_for_embedding, output_from_embedding, input_length=max_len_text))
# After the Embedding layer,
# our activations have shape `(samples, maxlen, 8)`.
# Output: (batch dimension, input_length, output_dim)

# Convolutional Layer 1D
# In text 
# Conv1D(
#    filters = dimensionality of the output space
#    kernel_size = window_size
#    stride = position jumps of the window (defaults to 1)
# )
initializer_large_model = initializers.RandomNormal(mean=0.0, stddev=0.02, seed=None)
model.add(layers.Conv1D(filters=conv_output_dim, kernel_size=kernel_size_7, activation=activation_conv, kernel_initializer=initializer_large_model))

# Max Pooling 1D
# MaxPool1D(
#    pool_size = window size
#    strides = by default equals pool_size, so each window doesn't overlap
# )
model.add(layers.MaxPool1D(pool_size=pool_size))
model.add(layers.Conv1D(filters=conv_output_dim, kernel_size=kernel_size_7, activation=activation_conv))
model.add(layers.MaxPool1D(pool_size=pool_size))
model.add(layers.Conv1D(filters=conv_output_dim, kernel_size=kernel_size_3, activation=activation_conv))
model.add(layers.Conv1D(filters=conv_output_dim, kernel_size=kernel_size_3, activation=activation_conv))
model.add(layers.Conv1D(filters=conv_output_dim, kernel_size=kernel_size_3, activation=activation_conv))
model.add(layers.Conv1D(filters=conv_output_dim, kernel_size=kernel_size_3, activation=activation_conv))
model.add(layers.MaxPool1D(pool_size=pool_size))

# Second part, full connected layers
initializer_large_model = initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation=activation_dens, kernel_initializer=initializer_large_model))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Dense(1024, activation=activation_dens))
model.add(layers.Dropout(dropout_rate))

# Last layer, according to problem to solve
# (just have 2 classes)
# But, it has to be though a Dense(2,...) , with Dense(1,...) not working so neither do activation "sigmoid"
model.add(layers.Dense(2, activation='softmax')) 


model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1014, 128)         8832      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1008, 256)         229632    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 336, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 330, 256)          459008    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 110, 256)          0         
_________________________________________________________________
conv1d_3 (Conv1D)    

In [6]:
# add optimizer and loss function and metrics to return 
# binary_crossentropy ? 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
def getAlphabetDict(alpha):
    d = {}; c = 1
    for a in alpha: d[a] = c; c += 1
    return d

In [8]:
def getDataAndClasses(tokenizer, info, alpha, max_len_text, batch_len = None, from_i = 0):
        
    lines = info['lines']
    labels = info['labels']
    if batch_len != None: 
        lines = lines[from_i:(from_i + batch_len)]
        labels = labels[from_i:(from_i + batch_len)]
    
    sequences = tokenizer.texts_to_sequences(lines)
    """EX: [[15, 9, 2, 5, 3, 1, 13, 12, 28, 1, 14, 17 ...""" # Each, number represents a letter in the alphabet
    data = preprocessing.sequence.pad_sequences(sequences, maxlen=max_len_text, padding='post')
    # Data of shape (quantity of info (sentences), max_len_text)
    """EX: array([[15,  9,  2, ...,  0,  0,  0],
       [ 6,  8,  2, ...,  0,  0,  0],
       [21,  5,  3, ...,  0,  0,  0]], dtype=int32)"""  
    
    classes = utils.to_categorical(labels, num_classes=len(info['total_labels']))
    """EX: array([[0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)"""
    
    return {'data': data, 'classes': classes}

In [11]:
hist_arr = []
def runByBatches(model, train_info, test_info, alpha, max_len_text, batch_size, test_batch_size, until = None):
    
    # Preprocess Data 
    tokenizer = preprocessing.text.Tokenizer(char_level=True, lower=True, filters=alpha) # , oov_token= ?  
    tokenizer.word_index = getAlphabetDict(alpha)
    """EX: {'a': 1,
         'b': 2,
         'c': 3 ... """
    
    # After each epoch , the test data is validated
    # test_data = getDataAndClasses(tokenizer, test_info, alpha, max_len_text)
    test_data = getDataAndClasses(tokenizer, test_info, alpha, max_len_text, test_batch_size,0)
        
    total_train_len = train_info['total_lines']
    total_tst_len = test_info['total_lines']
    count = count_tst = 0
    
    while count < total_train_len and (until != None and count < until):
        
        # Get elements
        size = batch_size if batch_size < (total_train_len - count) else total_train_len - count
        size_tst = test_batch_size if test_batch_size < (total_tst_len - count_tst) else total_tst_len - count_tst
        train_data = getDataAndClasses(tokenizer, train_info, alpha, max_len_text, size, count)
        #test_data = getDataAndClasses(tokenizer, test_info, alpha, max_len_text, size_tst, count_tst)
        print(f'-----  doing from trining/test: {count} - {size + count}') # / {count_tst} - {size_tst}')
        count += batch_size   
        count_tst += test_batch_size
        
        history = model.fit(train_data['data'], 
                            train_data['classes'],
                            validation_data=(test_data['data'], test_data['classes']),
                            epochs=100, # 5000
                            batch_size=128,
                            verbose=2
                           )
        hist_arr.append(history.history)
        
        train_data = None
        #test_data = None

In [12]:
def getWindowOfBatchInArray(a, from_i, to_i):
    tot_len = len(a)
    until_pos = from_i + to_i
    remaining = 0
    if until_pos > tot_len: 
        remaining = until_pos - tot_len 
        return a[from_i:(tot_len + 1)]+a[0:remaining]
    else:
        return a[from_i:to_i]

In [ ]:
test_batch_size = math.ceil(test_data_info['total_lines']/\
    (train_data_info['total_lines']/test_data_info['total_lines']))
test_size = 15000
train_size = 40000
n = 1
runByBatches(model, 
            train_data_info, 
            test_data_info, 
            alphabet, 
            max_len_text, 
            train_size, #test_data_info['total_lines'],
            test_size, # test batch size
            train_size * n # until n chunks of data (n repetitions)
           )


-----  doing from trining/test: 0 - 40000
Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 15000 samples
Epoch 1/100
 - 321s - loss: 0.6933 - acc: 0.5141 - val_loss: 0.6931 - val_acc: 0.5071
Epoch 2/100
 - 314s - loss: 0.6931 - acc: 0.5062 - val_loss: 0.6935 - val_acc: 0.4929
Epoch 3/100
 - 339s - loss: 0.6934 - acc: 0.5057 - val_loss: 0.6930 - val_acc: 0.5071
Epoch 4/100
 - 420s - loss: 0.6931 - acc: 0.5069 - val_loss: 0.6931 - val_acc: 0.5071
Epoch 5/100
 - 345s - loss: 0.6931 - acc: 0.5084 - val_loss: 0.6930 - val_acc: 0.5071
Epoch 6/100
 - 312s - loss: 0.6931 - acc: 0.5086 - val_loss: 0.6931 - val_acc: 0.5071
Epoch 7/100
 - 303s - loss: 0.6930 - acc: 0.5085 - val_loss: 0.6930 - val_acc: 0.5071
Epoch 8/100
 - 299s - loss: 0.6930 - acc: 0.5078 - val_loss: 0.6931 - val_acc: 0.5071
Epoch 9/100
 - 312s - loss: 0.6931 - acc: 0.5071 - val_loss: 0.6932 - val_acc: 0.5071
Epoch 10/100
 - 309s - loss: 0.6930 - acc: 0.5086 - val_loss: 0.6931 - val_acc: 0.5071

In [ ]:
"""tokenizer = preprocessing.text.Tokenizer(char_level=True, lower=True, filters=alphabet) # , oov_token= ?  
tokenizer.word_index = getAlphabetDict(alphabet)
t = getDataAndClasses(tokenizer, test_data_info, alphabet, max_len_text, 10000,0)"""

In [ ]:
"""hist_arr[0].history"""

In [ ]:
def plotHistoryList(h):
    acc = h['acc']
    val_acc = h['val_acc']
    loss = h['loss']
    val_loss = h['val_loss']

    epochs = range(1, len(acc) + 1)

    # "bo" is for "blue dot"
    plt.plot(epochs, loss, 'bo', label='Training loss')
    #plt.plot(epochs, acc, 'ro', label='Accuracy')
    # b is for "solid blue line"
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    #plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [ ]:
for h in hist_arr:
    plotHistoryList(h)

In [ ]:
# FROM FIRST TEST
"""
By using a sample data of 10000 each time, and 10000 for validation, 
it established that at epoch 3 it was enough since it started to have overfitting

[{'val_loss': [0.6950897953033447,
   0.6947383718490601,
   0.6934752422332764,
   0.6931729074478149,
   0.6935873050689697],
  'val_acc': [0.4875, 0.4875, 0.4875, 0.4875, 0.4875],
  'loss': [0.694396387386322,
   0.6932769330978393,
   0.6929676490783692,
   0.6933105537414551,
   0.6934012260437011],
  'acc': [0.5027, 0.51, 0.5084, 0.5055, 0.5029]},
 {'val_loss': [0.693684473323822,
   0.69328423204422,
   0.6938694007873535,
   0.693745785331726,
   0.693725270652771],
  'val_acc': [0.5125, 0.5125, 0.5125, 0.5125, 0.5125],
  'loss': [0.6909599201202392,
   0.6909819056510925,
   0.690941504573822,
   0.6910166868209839,
   0.6907023878097535],
  'acc': [0.5339, 0.5354, 0.5354, 0.5354, 0.5354]},
 {'val_loss': [0.6928669214248657,
   0.6928351986885071,
   0.6929179975509644,
   0.6928665622711182,
   0.6929092376708984],
  'val_acc': [0.5125, 0.5125, 0.5125, 0.5125, 0.5125],
  'loss': [0.6928800000190735,
   0.6927422769546508,
   0.692695544910431,
   0.6927323137283325,
   0.6927251628875732],
  'acc': [0.5171, 0.5172, 0.5171, 0.5171, 0.5157]}]

"""
# SECOND

"""
[{'val_loss': [0.6933708633422851,
   0.6928537891387939,
   0.692834661102295,
   0.6928840206146241,
   0.6929055606842041],
  'val_acc': [0.4875, 0.5125, 0.5125, 0.5125, 0.5125],
  'loss': [0.6934309574127198,
   0.6930742263793945,
   0.6930086311340332,
   0.6929674413681031,
   0.6930006246566772],
  'acc': [0.50965, 0.51025, 0.51005, 0.5117, 0.50945]},
 {'val_loss': [0.6928420051574707,
   0.6929439851760865,
   0.693077564239502,
   0.6930943966865539,
   0.6930167970657348],
  'val_acc': [0.5125, 0.5125, 0.5125, 0.5125, 0.5125],
  'loss': [0.6932257574081421,
   0.6932153765678406,
   0.6932126870155334,
   0.6931769281387329,
   0.693256987953186],
  'acc': [0.5025, 0.5051, 0.50615, 0.50025, 0.49795]}]
"""

# Third 30000 / 15000
"""
[{'val_loss': [0.6932847999890646,
   0.6933213699658711,
   0.6930976839383444,
   0.6931018094380696,
   0.6930792671521505],
  'val_acc': [0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279],
  'loss': [0.6933522695223491,
   0.6929458774248759,
   0.6930164001146952,
   0.6929120743115743,
   0.6929684819221497],
  'acc': [0.5136000000158946,
   0.5124333333333333,
   0.5129666666984558,
   0.5137000000317892,
   0.5138333333651225]},
 {'val_loss': [0.6931314949353536,
   0.6930487598737081,
   0.6931946491559347,
   0.693049590587616,
   0.6932759721120199],
  'val_acc': [0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279],
  'loss': [0.6929239110628764,
   0.6928764338493347,
   0.6929650987943013,
   0.6928932430585225,
   0.6928442873636882],
  'acc': [0.5129333333015442,
   0.5128,
   0.5133666666666666,
   0.5129999999841054,
   0.5129666666666667]}]
"""
# Fourth 100 000 / 15 000
"""
[{'val_loss': [0.6935770348866781,
   0.693049957370758,
   0.6931386842091878,
   0.6931923212369283,
   0.6930569999694824],
  'val_acc': [0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279],
  'loss': [0.693238242149353,
   0.6929247446441651,
   0.692936732673645,
   0.6928754871940613,
   0.6928619545555115],
  'acc': [0.5106, 0.512, 0.51234, 0.51267, 0.51267]}]
"""
# Fifth 20 000 / 15 000 (5 times)
"""
[{'val_loss': [0.6930875172615051,
   0.6930843497912089,
   0.6934041076978048,
   0.6935051878611247,
   0.6931250178019206],
  'val_acc': [0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279],
  'loss': [0.6936674816131592,
   0.6930888975143432,
   0.6951768466949463,
   0.693067194366455,
   0.6930339816093445],
  'acc': [0.50965, 0.50625, 0.51175, 0.51125, 0.5125]},
 {'val_loss': [0.6931213233629863,
   0.6931330739021301,
   0.6930682267824809,
   0.693061719640096,
   0.6931284649848938],
  'val_acc': [0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279],
  'loss': [0.6932584546089172,
   0.6931955748558044,
   0.6931753499984741,
   0.6931837162017822,
   0.6932215364456177],
  'acc': [0.5019, 0.50075, 0.50005, 0.50375, 0.5013]},
 {'val_loss': [0.6931777195930481,
   0.6935282015800476,
   0.6935493802388509,
   0.6932427165031433,
   0.6935413696289062],
  'val_acc': [0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279],
  'loss': [0.6924584628105164,
   0.6921570532798768,
   0.6920773757934571,
   0.692116604423523,
   0.6921314051628112],
  'acc': [0.52205, 0.5237, 0.5237, 0.5237, 0.5237]},
 {'val_loss': [0.6931134839375814,
   0.6930499877929688,
   0.6930544673919677,
   0.6930453857421875,
   0.6930874656677246],
  'val_acc': [0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279],
  'loss': [0.6930728750228882,
   0.692966617679596,
   0.6930215294837951,
   0.6930123390197754,
   0.6930175746917725],
  'acc': [0.50985, 0.5098, 0.50985, 0.50985, 0.5098]},
 {'val_loss': [0.693133440430959,
   0.6931064677874247,
   0.6930852600097657,
   0.6930747243245443,
   0.6930734438578288],
  'val_acc': [0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279,
   0.5071333333492279],
  'loss': [0.6928748257637024,
   0.6929110748291015,
   0.6928599676132202,
   0.6928945977210998,
   0.6928995547294616],
  'acc': [0.5126, 0.5126, 0.5126, 0.5126, 0.5126]}]
"""

In [ ]:
hist_arr